# Practical: PyProj, Shapely, GeoPandas, and Rasterio

## PyProj

PyProj is a python interface to PROJ, a cartographic projections and coordinate transformations library.
Using PyProj we can import a Coordinate Reference System object (CRS) which will allow us to get a reference
system in various way.

In [ ]:
from pyproj import CRS

For example to get the WGS 84, we can use its EPSG id:

In [ ]:
crs = CRS.from_epsg(4326)
crs

Or, using its full string:

In [ ]:
CRS.from_string("epsg:4326")

To transform coordinates from a CRS to another CRS we can use the Transform object of PyProj.

In [ ]:
from pyproj import Transformer

Once imported we can define a transformer object in different ways. For example, using CRS objects:

In [ ]:
wsg84 = CRS.from_epsg(4326)
osgb36 = CRS.from_epsg(27700)
transformer = Transformer.from_crs(wsg84, osgb36)

Or, by using EPSG ids:

In [ ]:
transformer = Transformer.from_crs(4326, 27700)

Or, by using EPSG strings:

In [ ]:
transformer = Transformer.from_crs("EPSG:4326", "EPSG:27700")

The `Transfomer` class implements a `transfom` method that allows the transformation of the coordinates in the first
CRS to the second CRS:

In [ ]:
transformer.transform(51.5074, 0.1278)

## Shapely

Shapely is a Python library for the manipulation and analysis of planar geometric objects. 


### Points

With Shapely we can easily create points. 

To create a point object in Shapely we first need to import its class from the geometry module:

In [ ]:
from shapely.geometry import Point

To create a point we need to pass to its constructor a pair of coordinate values:

In [ ]:
pt1 = Point(0, 0)

To get the coordinates of a point object we can use the `xy` attribute. 
This will return a tuple containing the x and y coordinates of the point as two arrays of length 1.

In [ ]:
pt1.xy

Given two points, we can compute their distance using the `distance` method available in every instance of
the `Geometry`  class and its children classes. The Point class is a child of the `Geometry` class.

In [ ]:
pt2 = Point(1, 1)
pt1.distance(pt2)

### LineString

`LineString`s are used to represent a series of segments all connect to each other. 

In order to use `LineString`s we first need to import the `LineString` class from the geometry module of Shapely:

In [ ]:
from shapely.geometry import LineString

If we want to create a segment, we just need to pass a list of two pair of coordinates to the `LineString` constructor:

In [ ]:
line = LineString([(0, 0), (1, 1)])
line

Or, we can pass a list of points:

In [ ]:
line = LineString([pt1, pt2])
line

To get the coordinates of this line, like for the point object, we can use the `xy` attribute. However, this time the
returning arrays of x and y values will have length 2:

In [ ]:
line.xy

Also LineStrings, as any geometric object, implements the distance operator. Following a non trivial example of the
distance between a point and a line:

In [ ]:
pt3 = Point(0.4, 0.5)
line.distance(pt3)

Think about how you would compute this distance manually.

We can create more complex `LineString`s by passing a longer list of points to the `LineString` constructor:

In [ ]:
line = LineString([pt1, pt2, (1, 0), (0.6, 0.4)])
line

## Polygon

In Shapely we can also define polygons. 

In [ ]:
from shapely.geometry import Polygon

To create a polygon we can pass the same list of coordinates passed to create the previous more complex `LineString`:

In [ ]:
polygon = Polygon([(0, 0), (1, 1), (1, 0), (0.6, 0.4)])
polygon

This will create a polygon with an external `LineString` defining its perimeter. To get this `LineString` back, we
need to use the `exterior` attribute:

In [ ]:
polygon.exterior

If we want to get the coordinates of this `LineString`, we can use the attribute `xy`:

In [ ]:
polygon.exterior.xy

If we now compare the points we have used to construct the polygon and this exterior `LineString`, we can observe
that there is an additional point. This point is added automatically by the `Polygon` constructor in order to assure
that the `LineString` is close.

### Plotting Shapely Shapes with Matplotlib

To plot Shapely shapes using Matplotlib we first need to import the object `matplotlib.pyplot`:

In [ ]:
import matplotlib.pyplot as plt

Then, we need to extract the coordinates from a Shapely object we wish to plot in Matplotlib:

In [ ]:
x, y = polygon.exterior.xy

At this point, we have all we need to plot in Matplotlib. We can now pass these coordinates to the `fill` method
of `plt`. Remember that we use `fill` because we are plotting a polygon. Use `plot` if you want to plot just
its perimeter.

In [ ]:
plt.fill(x, y)
plt.show()

### Attributes and Methods of Geometric Objects

Any geometric objects of Shapely provides a series of methods that are useful to compute various geometric
properties. These methods are all defined by the parent `Geometry` class. However, their behavior changes based
on their specific class.

For example, to compute the length of a shape:

In [ ]:
pt1.length, line.length, polygon.length

To compute the area of a shape:

In [ ]:
pt1.area, line.area, polygon.area

To get the geometric type of an object:

In [ ]:
pt1.geom_type, line.geom_type, polygon.geom_type

To compute the distance between two objects:

In [ ]:
pt3.distance(line), line.distance(polygon)

To compute the distance from the two furthest points of the objects:

In [ ]:
pt3.hausdorff_distance(line), line.hausdorff_distance(polygon)

### Topological Relationships

Using Shapely we can easily identify if two objects are touching (their contact point is at their border) or one
object is contained in the other one.

To check if two shapes are touching:

In [ ]:
polygon.touches(pt1)

To verify this, let's plot these two objects:

In [ ]:
x, y = polygon.exterior.xy
plt.fill(x, y)
x, y = pt1.xy
plt.plot(x, y, 'ro')
plt.show()

However, if we check whether the point is contained by the object we get:

In [ ]:
polygon.contains(pt1)

Looking at the previous plot, this is clearly correct since the point is on the border of the polygon.

### Geometric Operations

A buffering operation returns a polygon around an object that is offset by a given distance. 

Following some examples:

In [ ]:
pt1.buffer(1)

In [ ]:
line.buffer(1)

In [ ]:
polygon.buffer(1)

We can also perform set operations over polygons.

For example we can compute the intersection of a buffered point and a polygon:

In [ ]:
pt1.buffer(1).intersection(polygon)

### Using PyProj on Shapes

To calculate the geodesic length of a Shapely geometry we can use PyProj. To do this we first need to import the
Geod class of pyproj:

In [ ]:
from pyproj import Geod

Then, let's define two points, one with coordinates centered in London and one with coordinates centered in
Brighton both using the WSG 84 CRS:

In [ ]:
london = Point(51.5074, 0.1278) 
brighton = Point(50.8225, 0.1372)

We then define a `LineString` between these two points:

In [ ]:
line = LineString([Point(51.5074, 0.1278), Point(50.8225, 0.1372)])

Finally, we construct a Geod object with the WGS 84 CRS as a parameter, then use the `geometry_length` method to
compute their distance on this geode:

In [ ]:
geod = Geod(ellps="WGS84")
geod.geometry_length(line)

# Exercise 30

Use Shapely to solve the Point-In-Polygon problem. 

To solve this exercise you need: 
1. to download the CSV file describing the polygon used for the assignment;
2. define a polygon shape using Shapely;
3. get a point from the user;
3. test whether this point is inside, outside, or on the border of the polygon, and; 
4. plot the solution.

## GeoPandas

GeoPandas is an open source project to make working with geospatial data in Python easier.

Note that if we want to use the plotting functionality of GeoPandas also the package descartes needs to be installed.

GeoPandas is usually imported like this:

In [ ]:
import geopandas as gpd

GeoPandas extends the datatypes used by Pandas to allow spatial operations on geometric types. It extends DataFrame
into GeoDataFrame and Series into GeoSeries.

In the following we will work on a dataset of the GeoPandas package. We will now load the Natural Earth in Low
Resolution dataset:

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

On this dataset we can perform all the Pandas operations we have learned in the previous practical, like:

In [ ]:
world.head()

To get the type of each Series:

In [ ]:
world.dtypes

Here, we observe that this GeoDataFrame contains a "geometry" GeoSeries containing Shapely geometry objects.
GeoPandas supports any kind of Shapely geometry object.

A GeoDataFrame needs always to have a GeoSeries indicated as geometry. This is the GeoSeries on which
GeoPandas will perform the geospatial operations when called on a DataFrame. To know which column in the
GeoDataFrame is the column indicated as geometry we can type:

In [ ]:
world.geometry.name

To set another column as the geometry one, we need to type:

In [ ]:
world = world.set_geometry("geometry")

Any geospatial operation performed on a DataFrame will be always executed on the geometry GeoSeries.
For example, when calling plot, GeoPandas will plot the content of the GeoSeries indicated as geometry:

In [ ]:
world.plot()

To get the current CRS of this GeoDataFrame we can use the `crs` attribute:

In [ ]:
world.crs

Let's now select and plot the United Kingdom shape:

In [ ]:
world[world.name == "United Kingdom"].plot()

Does it look like what you expect?

To change CRS to the British National Grid we can use the `to_crs` method with the appropriate OSPG id:

In [ ]:
world.to_crs(27700)[world.name == "United Kingdom"].plot()

We can compute the area of each polygon:

In [ ]:
world.area

Or, their bounds:

In [ ]:
world.bounds

Or, their total bound:

In [ ]:
world.total_bounds

Or, compute the centroid (point) of each entry:

In [ ]:
world.centroid

Or, to compute the representative point of each entry:

In [ ]:
world.representative_point()

Or, return the geometric type of each entry:

In [ ]:
world.geom_type

Or, compute the distance between a point and each entry of the GeoDataFrame:

In [ ]:
pt0 = Point(0, 0)

world.distance(pt0)

# Exercise 31

Like Exercise 30, but this time use GeoPandas to test every point provided in the CSV file of the assignment.

## Rasterio

Rasterio is a library that allows you to work with raster data. Rasterio can manipulate various raster file formats
and provides a Python API based on NumPy arrays.

Rasterio is usually imported like this:

In [ ]:
import rasterio

In order to use Rasterio we need to load a GeoTiff. We will use the GeoTiff downloaded from Digimap representing the
Land Cover of UK sampled in 2015. More details about this dataset can be found at
this [link](https://digimap.edina.ac.uk/webhelp/environment/environmentdigimaphelp.htm#data_information/lcm2015.htm).

We can load this raster file like this:

In [ ]:
import os

dataset = rasterio.open(os.path.join('7 - Material', 'land_cover_map_2015_uk.tif'))

To know which CRS is currently in use:

In [ ]:
dataset.crs

To quickly have a look at this raster data we can use the method show:

In [ ]:
from rasterio import plot

rasterio.plot.show(dataset)

Raster data are stored in layers. To know how many layers this dataset contains, we can use the count attribute of
the dataset:

In [ ]:
dataset.count

To know how wide is this raster in pixels:

In [ ]:
dataset.width

To know how tall is this raster in pixels:

In [ ]:
dataset.height

To know the data type of this raster:

In [ ]:
dataset.dtypes

To know the bounds in the current CRS: 

In [ ]:
dataset.bounds

To access the value of this raster as a NumPy array, we can use the `read` method. Because a raster dataset can
have multiple bands (layers) we need to pass as a parameter the id of the layer we want to read:

In [ ]:
dataset.read(1)

# Exercise 32

Estimate the area (in $m^2$) of each land cover value for the UK.